MECA.MECA_MOVIMIENTOS
MECA.CAPF_CLIENTES
MECA.MECA_CARTERAS
MECA.MECA_CONCEPTOS
MECA.MECA_OPERACIONES
MECA.MECA_SERIES
MECA.MECA_SECTORES
MECA.MECA_EMISORAS

In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

# Get the data and create the output directory (Extraction)

In [6]:
path_dir_data = "/home/ale1726/proyects/datalake/clientes/data/productos/MECA/data"

In [ ]:
# Obtenemos la fecha actual para crear un directorio
# donde se guardaran los archivos de salida
date_now = datetime.now().strftime("%d_%m_%Y") 
path_dir_today =  os.path.join(path_dir_data, date_now)

# Creamos el directorio si no existe
os.makedirs(path_dir_today, exist_ok=True)

In [18]:
query_productos_meca = """ 
WITH CLIENTES_VIG AS (
    SELECT CFCL.CLI_CLAVE_FFON, CFCL.CLI_CLAVE, CFCL.CLI_NOMBRE, CTDC.TDC_DESCRIPCION AS TIPO_CLIENTE
	FROM MECA.CAPF_CLIENTES CFCL
	LEFT JOIN MECA.CAPF_TIPOS_DE_CLIENTES CTDC ON CFCL.CLI_TDC_CLAVE = CTDC.TDC_CLAVE
	WHERE CFCL.CLI_EST_CLAVE = 'VIG' AND CFCL.CLI_NOMBRE != '0'
), OPE_C1 AS (
	SELECT MO.*, MI.INS_DESCRIPCION INSTRUMENTO
	FROM MECA.MECA_OPERACIONES MO
	LEFT JOIN MECA.MECA_INSTRUMENTOS MI ON MO.OPE_INS_CLAVE = MI.INS_CLAVE
), OPE_C2 AS (
	SELECT OP.*, CAT.TMO_DESCRIPCION TIPO_MOVIMIENTO
	FROM OPE_C1 OP
	LEFT JOIN MECA.MECA_TIPOS_MOVIMIENTO CAT ON OP.OPE_TMO_CLAVE = CAT.TMO_CLAVE
), TAB_OPE AS(
	SELECT OPE_FOLIO, OPE_TYPE, OPE_INS_CLAVE ,INSTRUMENTO, OPE_SER_NUM_SERIE, OPE_SER_EMI_CLAVE, 
		OPE_TMO_CLAVE, TIPO_MOVIMIENTO, OPE_CLI_CLAVE, OPE_DIV_CLAVE_SWIFT, OPE_NUM_TITULOS, OPE_PRECIO_POSTURA
		OPE_IMPORTE_TITULOS, OPE_FECHA_PACTADA, OPE_FECHA_SALDADA, OPE_FECHA_ALTA, OPE_FECHA_VENCIMIENTO, OPE_TIPO_PRESTAMO,
		OPE_TIPO_CAMBIO, OPE_TASA_PRESTAMO, OPE_DESCUENTO_PREST, OPE_PLAZO, OPE_PORC_COMISION, OPE_PORC_IVA, OPE_IMPORTE_COMISION,
		OPE_IMPORTE_TOTAL, OPE_IMPORTE_PRIMA, OPE_PRECIO_PRESTAMO, OPE_PRECIO_PACTADA
	FROM OPE_C2
), TAB_FINAL AS (
	SELECT CV.*, TAOP.*
	FROM CLIENTES_VIG CV
	LEFT JOIN TAB_OPE TAOP ON TAOP.OPE_CLI_CLAVE = CV.CLI_CLAVE
) SELECT * FROM TAB_FINAL
"""

In [19]:
path_dir_productos = get_table(path_dir_today, db_meca, "productos_clientes_meca", query = query_productos_meca)

# Transform the data and output the transformation

In [55]:
path_dir_output_transformation = os.path.join("/home/ale1726/proyects/datalake/clientes/data/productos/MECA/agrupados", date_now)
os.makedirs(path_dir_output_transformation, exist_ok=True)
path_dir_output_transformation

'/home/ale1726/proyects/datalake/clientes/data/productos/MECA/agrupados/21_04_2025'

In [3]:
path_dir_output_transformation = '/home/ale1726/proyects/datalake/clientes/data/productos/MECA/agrupados/21_04_2025'

In [2]:
path_dir_data ='/home/ale1726/proyects/datalake/clientes/data/productos/MECA/agrupados/21_04_2025'

In [5]:
path_dir_productos = "/home/ale1726/proyects/datalake/clientes/data/productos/MECA/data/21_04_2025/productos_clientes_meca.dat"

In [6]:
productos_meca =  pd.read_csv(path_dir_productos, low_memory=False)

In [7]:
productos_meca["OPE_FECHA_ALTA"] = pd.to_datetime(productos_meca["OPE_FECHA_ALTA"])
productos_meca["OPE_FECHA_PACTADA"] = pd.to_datetime(productos_meca["OPE_FECHA_PACTADA"])
productos_meca["OPE_FECHA_SALDADA"] = pd.to_datetime(productos_meca["OPE_FECHA_SALDADA"])
productos_meca["OPE_FECHA_VENCIMIENTO"] = pd.to_datetime(productos_meca["OPE_FECHA_VENCIMIENTO"])

In [8]:
productos_meca["AÑO_FALTA"] = productos_meca["OPE_FECHA_ALTA"].dt.year.astype('Int64')
productos_meca["AÑO_FPACTADA"] = productos_meca["OPE_FECHA_PACTADA"].dt.year.astype('Int64')
productos_meca["AÑO_FSALDADA"] = productos_meca["OPE_FECHA_SALDADA"].dt.year.astype('Int64')
productos_meca["AÑO_FVENCIMIENTO"] = productos_meca["OPE_FECHA_VENCIMIENTO"].dt.year.astype('Int64')

productos_meca["MES_FALTA"] = productos_meca["OPE_FECHA_ALTA"].dt.month
productos_meca["MES_FALTA_STR"] = productos_meca["OPE_FECHA_ALTA"].dt.month_name()        
productos_meca["MES_FPACTADA"] = productos_meca["OPE_FECHA_PACTADA"].dt.month
productos_meca["MES_FSALDADA"] = productos_meca["OPE_FECHA_SALDADA"].dt.month
productos_meca["MES_FVENCIMIENTO"] = productos_meca["OPE_FECHA_VENCIMIENTO"].dt.month

#productos_meca["MES"] = productos_meca["OPE_FECHA_ALTA"].dt.month
#productos_meca["MES"] = productos_meca["OPE_FECHA_ALTA"].dt.month_name()        

In [94]:
agrupado_tipo_cliente = (
    productos_meca.groupby(['TIPO_CLIENTE'], as_index=False)
    .agg(
        valor=('OPE_FOLIO', 'count'), 
    ).sort_values(by=['valor'], ascending=True)
)

In [95]:
agrupado_tipo_cliente.to_csv(os.path.join(path_dir_output_transformation, "agrupado_tipo_cliente.csv"), index=False)

In [96]:
agrupado_portafolio = (
    productos_meca.groupby(['INSTRUMENTO'], as_index=False)
    .agg(
        valor=('OPE_FOLIO', 'count'), 
    ).sort_values(by=['INSTRUMENTO'], ascending=True)
)

In [97]:
agrupado_portafolio.to_csv(os.path.join(path_dir_output_transformation, "agrupado_portafolio.csv"), index=False)

In [14]:
agrupado_año_portafolio = (
    productos_meca.groupby(['AÑO_FALTA'], as_index=False)
    .agg(
        valor=('INSTRUMENTO', 'nunique'),
        instru = ('INSTRUMENTO', 'unique'), 
    ).sort_values(by=['valor'], ascending=False)
)

In [ ]:
agrupado_clt_anual = (
    productos_meca.groupby(['AÑO_FALTA'], as_index=False)
    .agg(
        valor=('INSTRUMENTO', 'nunique'),
        instru = ('INSTRUMENTO', 'unique'), 
    ).sort_values(by=['valor'], ascending=False)  
)

In [15]:
agrupado_año_portafolio

,AÑO_FALTA,valor,instru
0,1998,3,"[ACCIONES, ADS'S, ADR'S]"
2,2000,3,"[ACCIONES, ADS'S, ADR'S]"
1,1999,3,"[ACCIONES, ADS'S, ADR'S]"
9,2007,2,"[ACCIONES, ADS'S]"
13,2011,2,"[ADS'S, ACCIONES]"
12,2010,2,"[ACCIONES, ADS'S]"
11,2009,2,"[ACCIONES, ADS'S]"
10,2008,2,"[ACCIONES, ADS'S]"
14,2012,2,"[ACCIONES, ADS'S]"
8,2006,2,"[ACCIONES, ADS'S]"


In [16]:
agrupado_año_portafolio.to_csv(os.path.join(path_dir_output_transformation, "agrupado_año_portafolio.csv"), index=False)

In [17]:
agrupado_operaciones_anuales = (
    productos_meca.groupby(['AÑO_FALTA'], as_index=False)
    .agg(
        num_operaciones=('OPE_FOLIO', 'count'), 
        num_clts = ("CLI_NOMBRE","nunique"),
        monto_anuales_sum=('OPE_IMPORTE_TOTAL', 'sum'), 
        monto_anuales_min=('OPE_IMPORTE_TOTAL', 'min'), 
        monto_anuales_max=('OPE_IMPORTE_TOTAL', 'max'), 
        monto_anuales_mean=('OPE_IMPORTE_TOTAL', 'mean'), 
        monto_anuales_median=('OPE_IMPORTE_TOTAL', 'median')
    ).sort_values(by=['AÑO_FALTA'], ascending=True)
)

In [18]:
agrupado_operaciones_anuales.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales.csv"), index=False)

In [100]:
agrupado_operaciones_anuales_mes = (
    productos_meca.groupby(['AÑO_FALTA','MES_FALTA','MES_FALTA_STR'], as_index=False)
    .agg(
        num_operaciones=('OPE_FOLIO', 'count'), 
        monto_anuales_sum=('OPE_IMPORTE_TOTAL', 'sum'), 
        monto_anuales_min=('OPE_IMPORTE_TOTAL', 'min'), 
        monto_anuales_max=('OPE_IMPORTE_TOTAL', 'max'), 
        monto_anuales_mean=('OPE_IMPORTE_TOTAL', 'mean'), 
        monto_anuales_median=('OPE_IMPORTE_TOTAL', 'median')
    ).sort_values(by=['AÑO_FALTA','MES_FALTA'], ascending=True)
)

In [101]:
agrupado_operaciones_anuales_mes.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales_mes.csv"), index=False)

In [104]:
agrupado_clientes_año = (
    productos_meca.groupby(['AÑO_FALTA','CLI_CLAVE','CLI_NOMBRE', ], as_index=False)
    .agg(
        transacciones=('OPE_FOLIO', 'count'),
        num_productos=('INSTRUMENTO', 'nunique'),
        monto_mean=('OPE_IMPORTE_TOTAL', lambda x: round(x.mean(), 2) if x.mean() != 0 else 0),
    )
    .sort_values(by=['AÑO_FALTA'], ascending=False)
)


In [107]:
agrupado_clientes_año.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_año.csv"), index=False)

In [112]:
agrupado_clientes_tra_monto = (
    productos_meca.groupby(['CLI_NOMBRE'], as_index=False)
    .agg(
        transacciones=('OPE_FOLIO', 'count'),
        montos=('OPE_IMPORTE_TOTAL', lambda x: round(x.mean(), 2))
    )
    .sort_values(by='CLI_NOMBRE', ascending=False)
)

In [113]:
agrupado_clientes_tra_monto = agrupado_clientes_tra_monto[agrupado_clientes_tra_monto["transacciones"] > 1]
agrupado_clientes_tra_monto["montos"] = agrupado_clientes_tra_monto["montos"].fillna(0)

In [120]:
agrupado_clientes_tra_monto.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_tra_monto.csv"), index=False)

In [117]:
agrupado_clientes_tra_monto_año = (
    productos_meca.groupby(['AÑO_FALTA','CLI_NOMBRE'], as_index=False)
    .agg(
        transacciones=('OPE_FOLIO', 'count'),
        montos=('OPE_IMPORTE_TOTAL', lambda x: round(x.mean(), 2))
    )
    .sort_values(by='AÑO_FALTA', ascending=False)
)

In [ ]:
agrupado_clientes_tra_monto_año = agrupado_clientes_tra_monto_año[agrupado_clientes_tra_monto_año["transacciones"] > 1]
agrupado_clientes_tra_monto_año["montos"] = agrupado_clientes_tra_monto_año["montos"].fillna(0)

In [121]:
agrupado_clientes_tra_monto_año.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_tra_monto_año.csv"), index=False)

In [122]:
agrupado_año_seg_clt = (
    productos_meca.groupby(['AÑO_FALTA'], as_index=False)
    .agg(
        clientes=('CLI_NOMBRE', 'nunique'),
        clientes_unique=('CLI_NOMBRE', 'unique'),
    )
    .sort_values(by=['AÑO_FALTA'], ascending=False)
)

In [124]:
agrupado_año_seg_clt.to_csv(os.path.join(path_dir_output_transformation, "agrupado_año_seg_clt.csv"), index=False)